<a href="https://colab.research.google.com/github/dataSci-rigo/blink_detection/blob/master/Blinking_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def isBlinking(history, maxFrames):
    """ @history: A string containing the history of eyes status 
         where a '1' means that the eyes were closed and '0' open.
        @maxFrames: The maximal number of successive frames where an eye is closed """
    for i in range(maxFrames):
        pattern = '1' + '0' + '1'
        if pattern in history:
            return True
    return False

In [0]:
import cv2
face_detector= cv2.CascadeClassifier('drive/My Drive/w251/w251/haarcascade_frontalface_alt.xml')
open_eyes_detector= cv2.CascadeClassifier('drive/My Drive/w251/w251/haarcascade_eye_tree_eyeglasses.xml')
left_eye_detector= cv2.CascadeClassifier('drive/My Drive/w251/w251/haarcascade_lefteye_2splits.xml')
right_eye_detector= cv2.CascadeClassifier('drive/My Drive/w251/w251/haarcascade_righteye_2splits.xml')


In [0]:
def detect_and_display(face_detector, open_eyes_detector, left_eye_detector, right_eye_detector,images):
      eyes_detected=''
      for img in images:   
        #frame = video_capture.read()
        # resize the frame
        frame=img = cv2.imread(img)
        frame = cv2.resize(frame, (0, 0), fx=0.6, fy=0.6)

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Detect faces
        faces = face_detector.detectMultiScale(
            gray,
            scaleFactor=1.2,
            minNeighbors=5,
            minSize=(50, 50),
            flags=cv2.CASCADE_SCALE_IMAGE
        )

        # for each detected face
        for (x,y,w,h) in faces:
            # Encode the face into a 128-d embeddings vector


            # For now we don't know the person name
            name = "Unknown"

            # If there is at least one match:


            face = frame[y:y+h,x:x+w]
            gray_face = gray[y:y+h,x:x+w]

            eyes = []
            
            # Eyes detection
            # check first if eyes are open (with glasses taking into account)
            open_eyes_glasses = open_eyes_detector.detectMultiScale(
                gray_face,
                scaleFactor=1.1,
                minNeighbors=5,
                minSize=(30, 30),
                flags = cv2.CASCADE_SCALE_IMAGE
            )
            # if open_eyes_glasses detect eyes then they are open 
            if len(open_eyes_glasses) == 2:
                eyes_detected +='1'
                for (ex,ey,ew,eh) in open_eyes_glasses:
                    cv2.rectangle(face,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            
            # otherwise try detecting eyes using left and right_eye_detector
            # which can detect open and closed eyes                
            else:
                # separate the face into left and right sides
                left_face = frame[y:y+h, x+int(w/2):x+w]
                left_face_gray = gray[y:y+h, x+int(w/2):x+w]

                right_face = frame[y:y+h, x:x+int(w/2)]
                right_face_gray = gray[y:y+h, x:x+int(w/2)]

                # Detect the left eye
                left_eye = left_eye_detector.detectMultiScale(
                    left_face_gray,
                    scaleFactor=1.1,
                    minNeighbors=5,
                    minSize=(30, 30),
                    flags = cv2.CASCADE_SCALE_IMAGE
                )

                # Detect the right eye
                right_eye = right_eye_detector.detectMultiScale(
                    right_face_gray,
                    scaleFactor=1.1,
                    minNeighbors=5,
                    minSize=(30, 30),
                    flags = cv2.CASCADE_SCALE_IMAGE
                )

                eye_status = '0' # we suppose the eyes are open

                # For each eye check wether the eye is closed.
                # If one is closed we conclude the eyes are closed
                
                eyes_detected += eye_status

            # Each time, we check if the person has blinked
            # If yes, we display its name
           

      return  isBlinking(eyes_detected,3)
test1=['drive/My Drive/w251/w251/eye_open1.jpg','drive/My Drive/w251/w251/eye_closed.jpg','drive/My Drive/w251/w251/eye_open2.jpg']
test2=['drive/My Drive/w251/w251/eye_open1.jpg','drive/My Drive/w251/w251/eye_open2.jpg','drive/My Drive/w251/w251/eye_closed.jpg']
print(detect_and_display(face_detector, open_eyes_detector, left_eye_detector, right_eye_detector,test1))
print(detect_and_display(face_detector, open_eyes_detector, left_eye_detector, right_eye_detector,test2))

True
False
